In [4]:
import torch
import module_model
import lightning as L
import pennylane as qml
from pennylane import numpy as np
L.seed_everything(0)

Global seed set to 0


0

In [5]:
def encoding(num_ir_qubits, num_nr_qubits, mode):
    # rotation encoding on pennylane simulator
    def pennylane_encoding(_input, control_values):
        for i in range(num_nr_qubits):
            ctrl_H = qml.ctrl(qml.Hadamard, control=range(num_ir_qubits), control_values=control_values)
            ctrl_H(wires=num_ir_qubits+i)
            ctrl_R = qml.ctrl(qml.Rot, control=range(num_ir_qubits), control_values=control_values)
            # batch data
            if len(_input.shape) > 1:
                ctrl_R(theta=_input[:,0], phi=_input[:,1], omega=_input[:,2], wires=num_ir_qubits+i)
            # single data
            else:
                ctrl_R(theta=_input[0], phi=_input[1], omega=_input[2], wires=num_ir_qubits+i)

    # rotation encoding on qiskit
    num_wk_qubits = num_ir_qubits - 1
    def qiskit_encoding(_input, control_values):
        if len(_input.shape) > 1:
            theta, phi, omega = _input[:,0], _input[:,1], _input[:,2]
        # single data
        else:
            theta, phi, omega = _input[0], _input[1], _input[2]
        # see N.C. page.184
        for i in range(num_nr_qubits):
            # control values
            for q in range(num_ir_qubits):
                if control_values[q] == 0:
                    qml.PauliX(wires=q)
            # toffoli transformation
            if num_ir_qubits >= 2:
                qml.Toffoli(wires=(0, 1, num_ir_qubits))
            for q in range(num_wk_qubits-1):
                qml.Toffoli(wires=(2+q, num_ir_qubits+q, num_ir_qubits+1+q))
            # ctrl_H: decomposed by H = i Rx(pi) Ry(pi/2) (if complete graph with power of 2 nodes -> relative phase i becomes global)
            target_qubit = num_ir_qubits + num_wk_qubits + i
            qml.CRY(np.pi/2, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            qml.CRX(np.pi, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            # ctrl_R: Rot(phi, theta, omega) = Rz(omega) Ry(theta) Rz(phi)
            qml.CRZ(phi, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            qml.CRY(theta, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            qml.CRZ(omega, wires=(num_ir_qubits + num_wk_qubits - 1, target_qubit))
            # toffoli inverse transformation
            for q in reversed(range(num_wk_qubits-1)):
                qml.Toffoli(wires=(2+q, num_ir_qubits+q, num_ir_qubits+1+q))
            if num_ir_qubits >= 2:
                qml.Toffoli(wires=(0, 1, num_ir_qubits))
            # control values
            for q in range(num_ir_qubits):
                if control_values[q] == 0:
                    qml.PauliX(wires=q)

    if mode == "pennylane":
        return pennylane_encoding
    elif mode == "qiskit":
        return qiskit_encoding

In [6]:
IR = 3 # number of qubits in IR
NR = 2 # number of qubits in NR

for i in range(10):
    L.seed_everything(i) # to make the random initialized weights reproducible
    pennylane_circuit = module_model.QCGNN(IR, NR, 1, 1, ctrl_enc=encoding(IR, NR, "pennylane"))

    L.seed_everything(i) # to make the random initialized weights reproducible
    qiskit_circuit    = module_model.QCGNN(IR, NR, 1, 1, ctrl_enc=encoding(IR, NR, "qiskit"))

    batch_size = 1
    num_ptcs = np.random.randint(2**IR) + 1
    x = torch.rand(batch_size, num_ptcs, 3)

    y_pennylane = pennylane_circuit(x).reshape(-1)
    y_qiskit    = qiskit_circuit(x).reshape(-1)
    print(f"Pennylane:\n - {y_pennylane}")
    print(f"Qiskit:\n - {y_qiskit}")
    print(f"Difference norm = {torch.norm(y_pennylane-y_qiskit, p=2):.10f}")
    print(f"\n{'-'*100}\n")

Global seed set to 0
Global seed set to 0
/home/yianchen/.pyenv/versions/3.9.12/lib/python3.9/site-packages/pennylane/math/utils.py:227: UserWarning: Contains tensors of types {'autograd', 'torch'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(
Global seed set to 1
Global seed set to 1
Global seed set to 2
Global seed set to 2


# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([ 3.1391, -4.1633,  5.3773, -5.8651,  3.5260, -1.9418,  2.9362,  3.2477,
         4.6836, -1.9314,  4.1385, -4.1309,  3.5726, -5.6751,  1.4566, -0.3557],
       grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([ 3.1391, -4.1633,  5.3773, -5.8651,  3.5260, -1.9418,  2.9362,  3.2477,
         4.6836, -1.9314,  4.1385, -4.1309,  3.5726, -5.6751,  1.4566, -0.3557],
       grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000037570

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([-2.3010,  4.5080, -0.7773,  3.8009, -1.6603,  2.2806, -0.2871,  1.9528,
        -0.3428,  0.9435], grad_f

Global seed set to 3
Global seed set to 3
Global seed set to 4
Global seed set to 4
Global seed set to 5
Global seed set to 5


Pennylane:
 - tensor([-3.1551, -3.2923, -3.3829, -3.3523, -3.4551, -2.8127, -1.2576, -1.3178,
         1.3526, -0.5999, -3.2019, -3.1243], grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([-3.1551, -3.2923, -3.3829, -3.3523, -3.4551, -2.8127, -1.2576, -1.3178,
         1.3526, -0.5999, -3.2019, -3.1243], grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000017357

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([4.9203, 6.1964], grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([4.9203, 6.1964], grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000021325

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quan

Global seed set to 6
Global seed set to 6
Global seed set to 7
Global seed set to 7
Global seed set to 8
Global seed set to 8
Global seed set to 9


Pennylane:
 - tensor([ 0.0265, -4.1727, -5.8096, -6.9368, -0.7185, -4.9153, -5.9226, -5.5573,
        -7.1664, -7.9282, -5.9516, -6.4883, -2.1585, -4.3335, -6.1888, -6.3018],
       grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([ 0.0265, -4.1727, -5.8096, -6.9368, -0.7185, -4.9153, -5.9225, -5.5573,
        -7.1664, -7.9282, -5.9516, -6.4883, -2.1585, -4.3335, -6.1888, -6.3018],
       grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000051210

----------------------------------------------------------------------------------------------------

# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([-0.4749, -1.7244,  4.0903, -6.3153], grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([-0.4749, -1.7244,  4.0903, -6.3153], grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000007913

----------------------------------------------------------------------

Global seed set to 9


# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 0, 2)
# ModelLog: Quantum device  = default.qubit | Qubits (IR, WK, NR) = (3, 2, 2)
Pennylane:
 - tensor([ 2.5975, -3.4874, -0.1645,  0.6105,  2.3918, -2.7727,  1.5809, -2.4491,
         1.5957, -3.8615,  5.0309, -5.0276,  2.9914, -3.0150],
       grad_fn=<ReshapeAliasBackward0>)
Qiskit:
 - tensor([ 2.5975, -3.4874, -0.1645,  0.6105,  2.3918, -2.7727,  1.5809, -2.4491,
         1.5957, -3.8615,  5.0309, -5.0276,  2.9914, -3.0150],
       grad_fn=<ReshapeAliasBackward0>)
Difference norm = 0.0000022650

----------------------------------------------------------------------------------------------------

